In [1]:
cd D:\law_graph_agent

D:\law_graph_agent


In [8]:
from modules.retrieval_engine.hybrid_engine import HybridRetrievalEngine
from modules.retrieval_engine.dense_engine import DenseRetrievalEngine
from modules.retrieval_engine.bm25_engine import BM25RetrievalEngine

In [9]:
engine = DenseRetrievalEngine()

In [10]:
documents = [
            {
                'content': 'Article 1. Position and functions of the Government.',
                'metadata': {'article': 1}
            },
            {
                'content': 'Article 2. Organizational structure and members of the Government.',
                'metadata': {'article': 2}
            }
        ]

In [11]:
engine.index_documents(documents)

True

In [12]:
query = "Position"
results = engine.search(query, top_k=2)

In [13]:
results

(array([0, 1]), array([0.37579077, 0.21305636], dtype=float32))

In [ ]:
(array([0, 1]), array([0.37579077, 0.21305636], dtype=float32))